In [ ]:
!pip install numpy hist matplotlib mplhep pandas plotly scipy tqdm

In [30]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = [dir_path+"//data//"+"proAnubis_240716_2142.raw"] #list(map(lambda p: dir_path+"//data//"+p, data_list))[:1] # insert your file

In [31]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
import matplotlib.pyplot as plt

interval = 100 # Set your monitoring chunck size
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 50_000 # End the loop early
    
for file in file_path:
    fReader = rawFileReader.fileReader(file) # load in the classs object
    processedEvents = 0 # Initialisation
    noiseStart = 0
    firstNoise = True
    initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    mets = [[] for tdc in range(5)]
    file_name = file.split("//")[-1][:-4]
    tdc_mets = [[] for tdc in range(5)]
    Tot_TDC_info = [[] for tdc in range(5)]
    with tqdm(total=max_process_event_chunk, desc=f"Processing Events {file_name}", unit='Events') as pbar:
        while processedEvents < max_process_event_chunk:
            processedEvents += 1
            try:
                event_chunk, tdc_met, TDC_info = fReader.get_aligned_events(order=order, interval=interval, extract_tdc_mets = True) # get the aligned events
            except Exception as e:
                print(e)
                break
            [tdc_mets[i].append(tdc_met[i]) for i in range(5) if tdc_met[i] != 0]
            [Tot_TDC_info[i].extend(TDC_info[i]) for i in range(5) if TDC_info[i]]
        
            for idx, (i, j) in enumerate(order):
                x, y, l, m = ATools.find_tdc_alignment_metric(i, j) # determining which RPCs to use for aignment metric
                alignMet = ATools.calcAvgAlign(event_chunk, 0, x, y, l, m, i, j, processedEvents) # determine the alignment metric
                
                mets[idx].append(alignMet) # write to memory
                if alignMet > 40 and firstNoise:
                    noiseStart = processedEvents*interval
                    firstNoise = False
            pbar.update(1)
    fig, ax = plt.subplots(figsize=(10, 8))

    for idx, item in enumerate(order):
        met = mets[idx]
        i, j = item
        binsx = [x for x in range(len(met))]
        ax.plot(binsx, met, label=f'TDC{i} and TDC{j}, offset 0')
    ax.set_xlim(0, max_process_event_chunk)
    ax.set_ylim(-1, 40)
    ax.legend()
    ax.set_title(f'Alignment graph {file_name}')
    ax.set_ylabel('Average $\sqrt{d\eta^2+d\phi^2}$')
    ax.set_xlabel('Processed Event chunks')
    plt.savefig(f"output//transition//alignment_graph{file_name}.png")

    colors = ['blue', 'green', 'red', 'purple', 'orange']
    fig, ax = plt.subplots(figsize=(10, 8))
    for tdc in range(5):
        met = tdc_mets[tdc]
        binsx = [x*26.5 for x in range(len(met))]
        ax.plot(binsx,met, label = f'tdc {tdc}', color = colors[tdc])

    ax.set_xlim(0,max_process_event_chunk)
    ax.legend()
    ax.set_title(f'TDC monitoring metric {file_name}')
    ax.set_ylabel('bad time behavior / nominal time behavior')
    ax.set_xlabel('processed event chunks')
    plt.savefig(f"output//transition//tdc_monitoring{file_name}.png")
    plt.close()
    TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, noiseStart), (noiseStart, max_process_event_chunk*interval)], tdcs_to_plot=None, output_pdf=f'output//transition//TDC_channel{file_name}.pdf')


FileNotFoundError: [Errno 2] No such file or directory: 'proAnubis_240731_0217.raw'

In [16]:

fig, ax = plt.subplots(figsize=(10, 8))

for idx, item in enumerate(order):
    met = mets[idx]
    i, j = item
    binsx = [x for x in range(len(met))]
    ax.plot(binsx, met, label=f'TDC{i} and TDC{j}, offset 0')
ax.set_xlim(0, max_process_event_chunk)
ax.set_ylim(-1, 40)
ax.legend()

ax.set_title(f'Alignment graph {file_name}')
ax.set_ylabel('Average $\sqrt{d\eta^2+d\phi^2}$')
ax.set_xlabel('Processed Event chunks')
plt.savefig(f"output//transition//alignment_graph{file_name}.png")

colors = ['blue', 'green', 'red', 'purple', 'orange']
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    met = tdc_mets[tdc]
    binsx = [x*26.5 for x in range(len(met))]
    ax.plot(binsx,met, label = f'tdc {tdc}', color = colors[tdc])

ax.set_xlim(0,max_process_event_chunk)
ax.legend()
ax.set_title(f'TDC monitoring metric {file_name}')
ax.set_ylabel('bad time behavior / nominal time behavior')
ax.set_xlabel('processed event chunks')
plt.savefig(f"output//transition//tdc_monitoring{file_name}.png")
plt.close()

#TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, noiseStart), (noiseStart, max_process_event_chunk*interval)], tdcs_to_plot=None, output_pdf=f'output//transition//TDC_channel{file_name}.pdf')
